In [1]:
cd ../

/Users/sri/CS/Research/Stanford/research_2023/dsp


In [2]:
root_path = '.'

import os
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')
os.environ["DSP_CACHE_SQLITE_PATH"] = "cache.db"
os.environ["DSP_LOGGING_LEVEL"] = "10" # debug

import dsp
import openai

# openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'


lm = dsp.GPT3(model='gpt-3.5-turbo', model_type="chat")
rm = dsp.ColBERTv2(url=colbert_server)


In [3]:
from datetime import datetime, timedelta
import time

###### Case 1: 
- Example doesn't exists in the cache and the experiment end timestamp is not set to the future
- Expected:
    - Runs into an exception since it affects reproducibility. The example is missing in that timerange and should not be computed.

In [4]:
case1_cache_end_timerange = datetime.now().timestamp()

with dsp.settings.context(lm=lm, experiment_end_timestamp=case1_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

Exception: Cache does not exist for the given experiment timerange of between 1970-01-01T05:30:00 and 2023-07-02T16:34:08.000838.

###### Case 2: 
- Example doesn't exists in the cache and the experiment end timestamp is set to the future (by default)
- Expected:
    - The experiment is not trying to reproduce but create new reproducible results. Therefore, it is allowed to re-compute

In [5]:

with dsp.settings.context(lm=lm):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

2023-07-02 16:34:09,352 - DSP - DEBUG - Could not find a succesful experiment result in the cache for timerange between 1970-01-01T05:30:00 and future. Computing!
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Synchronized swimming was considered a valid Olympic sport in the year 1984.",
        "role": "assistant"
      }
    }
  ],
  "created": 1688063841,
  "id": "chatcmpl-7Wqo52s2WAoYZ1ucJbcDFbATQSJb6",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 26,
    "total_tokens": 42
  }
}


###### Case 3: 
- Example already exists in the cache within the specified timerange
- Expected:
    - Returns the results of the example without re-computing

In [6]:
case3_cache_end_timerange = (datetime.now() + timedelta(days=2)).timestamp()

with dsp.settings.context(lm=lm, experiment_end_timestamp=case3_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

2023-07-02 16:34:10,451 - DSP - DEBUG - Cached experiment result found between 1970-01-01T05:30:00 and 2023-07-04T16:34:10.449251. Retrieving result from cache.
{'id': 'chatcmpl-7Wqo52s2WAoYZ1ucJbcDFbATQSJb6', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'Synchronized swimming was considered a valid Olympic sport in the year 1984.', 'role': 'assistant'}}], 'created': 1688063841, 'model': 'gpt-3.5-turbo-0613', 'object': 'chat.completion', 'usage': {'completion_tokens': 16, 'prompt_tokens': 26, 'total_tokens': 42}}


###### Case 4: 
- There exists an example specified in the timerange but it failed for some reason.
- Expected:
    - Returns the exact error that occured, for the sake of reproducibility

In [7]:
# Let's intentionally make it fail
case4_cache_end_timerange = (datetime.now() + timedelta(seconds=3)).timestamp()
try:
    with dsp.settings.context(lm=lm, experiment_end_timestamp=case4_cache_end_timerange):
        answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:", dummy_kwargs="sample")
finally:
    time.sleep(3)

2023-07-02 16:34:11,674 - DSP - DEBUG - Could not find a succesful experiment result in the cache for timerange between 1970-01-01T05:30:00 and 2023-07-02T16:34:14.670129. Computing!


InvalidRequestError: Unrecognized request argument supplied: dummy_kwargs

In [8]:
# Should fail with the same exception
case4_cache_end_timerange = datetime.now().timestamp()

with dsp.settings.context(lm=lm, experiment_end_timestamp=case4_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:", dummy_kwargs="sample")

2023-07-02 16:34:16,619 - DSP - DEBUG - Failed operation found in the cache for experiment timerange between 1970-01-01T05:30:00 and 2023-07-02T16:34:16.617984. Raising the same exception.


Exception: Traceback (most recent call last):

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/utils/cache.py", line 334, in wrapper
    result = func(*args, **kwargs)

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/modules/gpt3.py", line 212, in cached_gpt3_turbo_request
    return _cached_gpt3_turbo_request_v2_wrapped(**kwargs)

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 594, in __call__
    return self._cached_call(args, kwargs)[0]

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 537, in _cached_call
    out, metadata = self.call(*args, **kwargs)

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 779, in call
    output = self.func(*args, **kwargs)

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/modules/gpt3.py", line 207, in _cached_gpt3_turbo_request_v2_wrapped
    return _cached_gpt3_turbo_request_v2(**kwar

# concurrency test

In [4]:
import threading
case5_cache_end_timerange = (datetime.now() + timedelta(days=2)).timestamp()

    
def run_snippet():
    with dsp.settings.context(lm=lm, experiment_end_timestamp=case5_cache_end_timerange):
        answer = dsp.settings.lm.basic_request(prompt="Q: what war did Archimedes invent war machines for?\nA:")

threads = []
for _ in range(10):
    thread = threading.Thread(target=run_snippet)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()


2023-07-02 20:13:49,922 - DSP - DEBUG - Could not find a succesful experiment result in the cache for timerange between 1970-01-01T05:30:00 and 2023-07-04T20:13:49.914930. Computing!
2023-07-02 20:13:49,923 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,923 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,925 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,927 - DSP - DEBUG - Could not find a succesful experiment result in the cache for timerange between 1970-01-01T05:30:00 and 2023-07-04T20:13:49.914930. Computing!
2023-07-02 20:13:49,927 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,928 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,929 - DSP - DEBUG - Operation is pending in the cache. Polling for result.
2023-07-02 20:13:49,931 - DSP - DEBUG - Operation is pending in 